# *Imports*

In [1]:
import os, json, math, time, hashlib
import numpy as np
import pandas as pd
import requests
from pinecone import Pinecone, ServerlessSpec
import os
from dotenv import load_dotenv
load_dotenv()

True

# *Configurations*

In [2]:
BASE_URL = "https://api.llmod.ai/v1"  
EMBED_MODEL = "RPRTHPB-text-embedding-3-small"
EMBED_DIMS = 1536

In [ ]:
# Chunking restrictions
MAX_TOKENS = 1024
MAX_OVERLAP_RATIO = 0.2
EMBED_DIMS = 1536

# Heuristic since we may not have a tokenizer:
# rough token ≈ word * 1.3, so max_words ~ MAX_TOKENS / 1.3
MAX_WORDS = int(MAX_TOKENS / 1.3)
OVERLAP_WORDS = int(MAX_WORDS * MAX_OVERLAP_RATIO)

START_WITH_N_TALKS = 5  # set None to do all data

# Output artifact files (cached embeddings)
OUT_META_JSONL = "ted_chunks_meta.jsonl"
OUT_EMB_NPY = "ted_chunks_embeds.npy"
OUT_IDMAP_JSON = "ted_chunks_idmap.json"  # to avoid duplicates across runs

# API keys
LLMOD_API_KEY = os.getenv("LLMOD_API_KEY")  # or paste here: "sk-...."
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key=PINECONE_API_KEY)

HEADERS = {"Authorization": f"Bearer {LLMOD_API_KEY}", "Content-Type": "application/json"}
INDEX_NAME = "ted"

# *Chunks*

In [4]:
def stable_hash(text: str) -> str:
    return hashlib.sha256(text.encode("utf-8")).hexdigest()

def approx_word_chunks(text: str, max_words: int, overlap_words: int):
    """
    Word-based chunker to approximate token limits.
    Ensures overlap <= 30% by construction if overlap_words <= 0.3*max_words.
    """
    words = text.split()
    if not words:
        return []

    chunks = []
    step = max_words - overlap_words
    if step <= 0:
        raise ValueError("overlap_words too large; step must be > 0.")

    start = 0
    while start < len(words):
        end = min(start + max_words, len(words))
        chunk_words = words[start:end]
        chunk_text = " ".join(chunk_words).strip()
        if chunk_text:
            chunks.append(chunk_text)
        if end == len(words):
            break
        start += step
    return chunks

def embed_texts_batch(texts, model=EMBED_MODEL, dims=EMBED_DIMS, max_retries=6):
    """
    Calls llmod.ai embeddings endpoint (OpenAI-compatible).
    Uses exponential backoff on transient errors.
    """
    url = f"{BASE_URL}/embeddings"
    payload = {
        "model": model,
        "input": texts,
        "dimensions": dims,  # keep aligned with the model default (1536)
    }

    for attempt in range(max_retries):
        try:
            r = requests.post(url, headers=HEADERS, data=json.dumps(payload), timeout=60)
            if r.status_code == 200:
                data = r.json()
                # OpenAI-style: data["data"] is list of {embedding: [...]}
                embs = [np.array(item["embedding"], dtype=np.float32) for item in data["data"]]
                return np.vstack(embs)
            # Retry on rate limit / transient server issues
            if r.status_code in (429, 500, 502, 503, 504):
                sleep_s = min(2 ** attempt, 30)
                time.sleep(sleep_s)
                continue
            # Otherwise: hard fail with details
            raise RuntimeError(f"Embeddings error {r.status_code}: {r.text[:500]}")
        except requests.RequestException as e:
            sleep_s = min(2 ** attempt, 30)
            time.sleep(sleep_s)
            last_err = e
    raise RuntimeError(f"Embeddings failed after retries. Last error: {last_err}")

In [5]:
# 1) Load the dataset CSV
# If your dataset is large, consider reading only needed columns first.
# Example assumes you already have it locally as ted.csv:
CSV_PATH = "ted_talks_en.csv"  # <-- change this

df = pd.read_csv(CSV_PATH)

# Optional: start small for budget
if START_WITH_N_TALKS is not None:
    df = df.head(START_WITH_N_TALKS).copy()

df.columns

Index(['talk_id', 'title', 'speaker_1', 'all_speakers', 'occupations',
       'about_speakers', 'views', 'recorded_date', 'published_date', 'event',
       'native_lang', 'available_lang', 'comments', 'duration', 'topics',
       'related_talks', 'url', 'description', 'transcript'],
      dtype='object')

In [6]:
# 2) Build chunk records (metadata + text)
records = []
for _, row in df.iterrows():
    talk_id = str(row.get("talk_id", ""))
    title = str(row.get("title", ""))
    transcript = str(row.get("transcript", "") or "")
    if not transcript.strip():
        continue

    chunks = approx_word_chunks(transcript, max_words=MAX_WORDS, overlap_words=OVERLAP_WORDS)

    for ci, chunk_text in enumerate(chunks):
        chunk_uid = stable_hash(talk_id + "|" + title + "|" + str(ci) + "|" + chunk_text)
        records.append({
            "chunk_uid": chunk_uid,
            "talk_id": talk_id,
            "title": title,
            "chunk_id": ci,
            "text": chunk_text,
        })

len(records), records[0].keys() if records else None

(14, dict_keys(['chunk_uid', 'talk_id', 'title', 'chunk_id', 'text']))

In [7]:
# 3) Load previous cache (so we don't re-embed)
if os.path.exists(OUT_IDMAP_JSON):
    with open(OUT_IDMAP_JSON, "r", encoding="utf-8") as f:
        seen = set(json.load(f))
else:
    seen = set()

new_records = [r for r in records if r["chunk_uid"] not in seen]
print("Total records:", len(records))
print("Already embedded:", len(records) - len(new_records))
print("To embed now:", len(new_records))

Total records: 14
Already embedded: 14
To embed now: 0


In [8]:
# 4) Embed new chunks in batches + append to cache
BATCH_SIZE = 64  # you can tune; smaller batches can be safer

# Load existing embeddings if present
if os.path.exists(OUT_EMB_NPY):
    old_embs = np.load(OUT_EMB_NPY)
else:
    old_embs = None

new_emb_list = []
to_write_meta = []

for i in range(0, len(new_records), BATCH_SIZE):
    batch = new_records[i:i+BATCH_SIZE]
    texts = [b["text"] for b in batch]
    embs = embed_texts_batch(texts)

    # Sanity check dimensions
    if embs.shape[1] != EMBED_DIMS:
        raise ValueError(f"Unexpected embedding dims: {embs.shape[1]} (expected {EMBED_DIMS})")

    new_emb_list.append(embs)
    to_write_meta.extend(batch)

    print(f"Embedded {min(i+BATCH_SIZE, len(new_records))}/{len(new_records)}")

# Append embeddings
if new_emb_list:
    new_embs = np.vstack(new_emb_list)
    all_embs = new_embs if old_embs is None else np.vstack([old_embs, new_embs])
else:
    all_embs = old_embs if old_embs is not None else np.zeros((0, EMBED_DIMS), dtype=np.float32)

# Save embeddings matrix
np.save(OUT_EMB_NPY, all_embs)

# Append metadata (jsonl)
if to_write_meta:
    with open(OUT_META_JSONL, "a", encoding="utf-8") as f:
        for r in to_write_meta:
            f.write(json.dumps({
                "chunk_uid": r["chunk_uid"],
                "talk_id": r["talk_id"],
                "title": r["title"],
                "chunk_id": r["chunk_id"],
                "text": r["text"],
            }, ensure_ascii=False) + "\n")

# Update seen set and save
for r in to_write_meta:
    seen.add(r["chunk_uid"])
with open(OUT_IDMAP_JSON, "w", encoding="utf-8") as f:
    json.dump(sorted(list(seen)), f)

print("Done.")
print("Embeddings shape:", all_embs.shape)
print("Saved:", OUT_EMB_NPY, OUT_META_JSONL, OUT_IDMAP_JSON)


Done.
Embeddings shape: (14, 1536)
Saved: ted_chunks_embeds.npy ted_chunks_meta.jsonl ted_chunks_idmap.json


# Pinecone Index

In [9]:
existing = [idx["name"] for idx in pc.list_indexes()]

if INDEX_NAME not in existing:
    pc.create_index(
        name=INDEX_NAME,
        dimension=EMBED_DIMS, # Ensuring using the correct dimensions (as returned by embedding model)
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1",
        )
    )

index = pc.Index(INDEX_NAME)
print("Ready:", INDEX_NAME)

Ready: ted


In [10]:
# ====== CONTINUE HERE: Upsert cached vectors into Pinecone ======

OUT_PINECONE_IDMAP_JSON = "pinecone_upserted_ids.json"

# Load cached embeddings
if not os.path.exists(OUT_EMB_NPY):
    raise FileNotFoundError(f"Missing embeddings file: {OUT_EMB_NPY}")
embs = np.load(OUT_EMB_NPY)

# Load cached metadata jsonl (must be same order as embeddings were appended)
if not os.path.exists(OUT_META_JSONL):
    raise FileNotFoundError(f"Missing metadata file: {OUT_META_JSONL}")

meta_rows = []
with open(OUT_META_JSONL, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if line:
            meta_rows.append(json.loads(line))

if len(meta_rows) != embs.shape[0]:
    raise ValueError(
        f"Mismatch: meta rows={len(meta_rows)} vs embeddings={embs.shape[0]}. "
        "These must be aligned (same append order)."
    )

print("Loaded embeddings:", embs.shape)
print("Loaded meta rows:", len(meta_rows))

# Load already-upserted ids (local cache to avoid re-upserting)
if os.path.exists(OUT_PINECONE_IDMAP_JSON):
    with open(OUT_PINECONE_IDMAP_JSON, "r", encoding="utf-8") as f:
        upserted = set(json.load(f))
else:
    upserted = set()

print("Already upserted IDs (local cache):", len(upserted))

def safe_metadata(m: dict) -> dict:
    """
    Pinecone metadata must be JSON-serializable, typically simple types.
    Also keep text size reasonable (metadata size limits exist).
    """
    text = m.get("text", "") or ""
    # Keep a snippet to avoid oversized metadata; adjust if you want.
    text_snippet = text[:2000]

    return {
        "talk_id": str(m.get("talk_id", "")),
        "title": str(m.get("title", "")),
        "chunk_id": int(m.get("chunk_id", 0)),
        "text": text_snippet,
    }

# Build items to upsert (skip already upserted IDs)
items = []
for i, m in enumerate(meta_rows):
    _id = m["chunk_uid"]
    if _id in upserted:
        continue

    vec = embs[i]
    if vec.shape[0] != EMBED_DIMS:
        raise ValueError(f"Bad dims at row {i}: got {vec.shape[0]} expected {EMBED_DIMS}")

    items.append((_id, vec.tolist(), safe_metadata(m)))

print("To upsert now:", len(items))

# Upsert in batches
UPSERT_BATCH = 100  # safe default; you can set 50 if you prefer
for start in range(0, len(items), UPSERT_BATCH):
    batch = items[start:start+UPSERT_BATCH]
    index.upsert(vectors=batch)

    # update local cache
    for _id, _, _ in batch:
        upserted.add(_id)

    if (start // UPSERT_BATCH) % 5 == 0:
        print(f"Upserted {min(start+UPSERT_BATCH, len(items))}/{len(items)}")

# Persist local upsert cache
with open(OUT_PINECONE_IDMAP_JSON, "w", encoding="utf-8") as f:
    json.dump(sorted(list(upserted)), f)

print("✅ Upsert complete.")
print("Local upsert cache saved to:", OUT_PINECONE_IDMAP_JSON)

# Optional: quick sanity check by describing index stats
try:
    stats = index.describe_index_stats()
    print("Index stats:", stats)
except Exception as e:
    print("describe_index_stats failed (non-fatal):", repr(e))


Loaded embeddings: (14, 1536)
Loaded meta rows: 14
Already upserted IDs (local cache): 14
To upsert now: 0
✅ Upsert complete.
Local upsert cache saved to: pinecone_upserted_ids.json
Index stats: {'_response_info': {'raw_headers': {'connection': 'keep-alive',
                                    'content-length': '184',
                                    'content-type': 'application/json',
                                    'date': 'Sat, 27 Dec 2025 13:52:06 GMT',
                                    'grpc-status': '0',
                                    'server': 'envoy',
                                    'x-envoy-upstream-service-time': '93',
                                    'x-pinecone-request-id': '538027401554865195',
                                    'x-pinecone-request-latency-ms': '92',
                                    'x-pinecone-response-duration-ms': '94'}},
 'dimension': 1536,
 'index_fullness': 0.0,
 'memoryFullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'__

# Rag

In [ ]:
# ====== RAG PIPELINE (Query -> Retrieve -> Answer) ======

import textwrap

CHAT_MODEL = "RPRTHPB-gpt-5-mini"

# Must choose & report (per assignment)
RAG_CHUNK_SIZE_TOKENS = MAX_TOKENS          # 2048 (max allowed)
RAG_OVERLAP_RATIO = MAX_OVERLAP_RATIO       # 0.30 (max allowed)
RAG_TOP_K = 2                               # <= 30 (tune: 5-12 often good)

assert RAG_CHUNK_SIZE_TOKENS <= 2048
assert RAG_OVERLAP_RATIO <= 0.30
assert 1 <= RAG_TOP_K <= 30

RETRIEVE_INCLUDE_TEXT = True  # we stored snippet in metadata["text"]

REQUIRED_SYSTEM_PROMPT = """You are a TED Talk assistant that answers questions strictly and
only based on the TED dataset context provided to you (metadata
and transcript passages). You must not use any external
knowledge, the open internet, or information that is not explicitly
contained in the retrieved context. If the answer cannot be
determined from the provided context, respond: “I don’t know
based on the provided TED data.” Always explain your answer
using the given context, quoting or paraphrasing the relevant
transcript or metadata when helpful.
"""

def embed_query(text: str) -> np.ndarray:
    """Embed one query text -> (1536,) float32"""
    emb = embed_texts_batch([text], model=EMBED_MODEL, dims=EMBED_DIMS)
    if emb.shape != (1, EMBED_DIMS):
        raise ValueError(f"Bad query embedding shape: {emb.shape}")
    return emb[0].astype(np.float32)

def retrieve_from_pinecone(query: str, top_k: int = RAG_TOP_K):
    """
    Returns list of matches with fields: id, score, metadata.
    """
    qvec = embed_query(query)
    res = index.query(
        vector=qvec.tolist(),
        top_k=top_k,
        include_metadata=True
    )
    # Pinecone returns an object/dict-like; normalize:
    matches = res.get("matches", res["matches"]) if isinstance(res, dict) else res.matches
    # print("Returned from PINECONE: \n", matches)
    out = []
    for m in matches:
        # handle dict-like or object-like
        mid = m.get("id", None) if isinstance(m, dict) else m.id
        score = m.get("score", None) if isinstance(m, dict) else m.score
        meta = m.get("metadata", {}) if isinstance(m, dict) else (m.metadata or {})
        out.append({"id": mid, "score": float(score) if score is not None else None, "metadata": meta})
    return out

def build_context(matches, max_chars: int = 12000) -> str:
    """
    Build a compact context block from retrieved matches.
    We cap total chars to avoid bloating the model context (efficiency).
    """
    parts = []
    total = 0

    for rank, item in enumerate(matches, start=1):
        md = item["metadata"] or {}
        title = str(md.get("title", ""))
        talk_id = str(md.get("talk_id", ""))
        chunk_id = md.get("chunk_id", "")
        text = str(md.get("text", "")) if RETRIEVE_INCLUDE_TEXT else ""

        block = (
            f"[{rank}] talk_id={talk_id} | title={title} | chunk_id={chunk_id} | score={item['score']:.4f}\n"
            f"PASSAGE:\n{text}\n"
        )

        if total + len(block) > max_chars:
            break
        parts.append(block)
        total += len(block)

    return "\n---\n".join(parts).strip()

def print_retrieved(matches, text_chars: int = 400):
    print("\n=== RETRIEVED MATCHES ===")
    for i, m in enumerate(matches, start=1):
        md = m.get("metadata", {}) or {}
        title = md.get("title", "")
        talk_id = md.get("talk_id", "")
        chunk_id = md.get("chunk_id", "")
        score = m.get("score", None)

        passage = (md.get("text", "") or "")
        passage = passage[:text_chars] + ("..." if len(passage) > text_chars else "")

        print(f"\n[{i}] score={score:.4f} | talk_id={talk_id} | chunk_id={chunk_id} | title={title}")
        print("PASSAGE:", passage)


def call_chat_model(question: str, context: str) -> str:
    """
    Calls llmod.ai chat completions endpoint (OpenAI-compatible).
    """
    url = f"{BASE_URL}/chat/completions"
    messages = [
        {"role": "system", "content": REQUIRED_SYSTEM_PROMPT},
        {"role": "user", "content": f"TED DATA CONTEXT:\n{context}\n\nQUESTION:\n{question}"},
    ]
    payload = {
        "model": CHAT_MODEL,
        "messages": messages,
    }

    # minimal retry/backoff
    max_retries = 6
    last_err = None
    for attempt in range(max_retries):
        try:
            r = requests.post(url, headers=HEADERS, data=json.dumps(payload), timeout=90)
            if r.status_code == 200:
                data = r.json()
                return data["choices"][0]["message"]["content"]
            if r.status_code in (429, 500, 502, 503, 504):
                time.sleep(min(2 ** attempt, 30))
                continue
            raise RuntimeError(f"Chat error {r.status_code}: {r.text[:500]}")
        except requests.RequestException as e:
            last_err = e
            time.sleep(min(2 ** attempt, 30))
    raise RuntimeError(f"Chat failed after retries. Last error: {last_err}")

def rag_answer(question: str, top_k: int = RAG_TOP_K, min_matches: int = 1, debug: bool = True):
    """
    End-to-end RAG: retrieve -> build context -> ask model.
    """
    matches = retrieve_from_pinecone(question, top_k=top_k)
    if debug:
        print(f"Retrieved: {len(matches)} matches (top_k={top_k})")
        print_retrieved(matches, text_chars=600)
    
    if debug:
        print(f"Retrieved: {len(matches)} matches (top_k={top_k})")

    if len(matches) < min_matches:
        return "I don’t know based on the provided TED data."

    context = build_context(matches)
    if debug:
        print("\nContext preview (first 600 chars):")
        print(context[:600] + ("..." if len(context) > 600 else ""))
        print("\n---\nRAG hyperparameters to report:")
        print(f"chunk_size_tokens={RAG_CHUNK_SIZE_TOKENS}, overlap_ratio={RAG_OVERLAP_RATIO}, top_k={top_k}")

    answer = call_chat_model(question, context)
    return answer

In [20]:
# ====== QUICK TEST ======
q = "What is the main message of the talk about education and creativity?"
print(rag_answer(q, top_k=RAG_TOP_K, debug=True))

Returned from PINECONE: 
 [{'id': '8fba17446ff938c960e0cdb07c4541dc19a74e071827f48e8dcc1b5b0a729aa5',
 'metadata': {'chunk_id': 0,
              'talk_id': '66',
              'text': "Good morning. How are you? (Audience) Good. It's been "
                      "great, hasn't it? I've been blown away by the whole "
                      "thing. In fact, I'm leaving. (Laughter) There have been "
                      'three themes running through the conference, which are '
                      'relevant to what I want to talk about. One is the '
                      'extraordinary evidence of human creativity in all of '
                      "the presentations that we've had and in all of the "
                      'people here; just the variety of it and the range of '
                      "it. The second is that it's put us in a place where we "
                      "have no idea what's going to happen in terms of the "
                      'future. No idea how this may play 